In [22]:
import requests
import json
import numpy as np
API_KEY = "AVMHeV6pxPlMksujAClTZYB9ylNpKnKdJWLUOVDogtwz"


In [21]:
def get_stories(x):
    BASE_URL = "https://www.wattpad.com/v4/stories?limit=100offset%3D0&offset="+str(x)

    header = {
    "Authorization": "Basic {}".format(API_KEY),
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36",

    }

    req = requests.get(BASE_URL.format("stories"), headers=header)
    json_response = req.json()
    return(json_response)
    #print(json.dumps(json_response, indent=3, sort_keys=True)) 

In [ ]:
#number of stories
N=10000
json_list=[]
for x in np.arange(0,N,100):
    json_list.append(get_stories(x))